In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
import sklearn 
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import catboost
import lightgbm
import xgboost
import json 
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
train = pd.read_csv('data/dota2_skill_train.csv', index_col='id')
test = pd.read_csv('data/dota2_skill_test.csv', index_col='id')

heroes = pd.read_csv('data/dota2_heroes.csv', index_col='hero_id')
abilities = pd.read_csv('data/dota2_abilities.csv', index_col='ability_id')
items = pd.read_csv('data/dota2_items.csv', index_col='item_id')

In [7]:
records = []
import json
i = 0
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        records.append(record)
        i+=1
        if i == 500:
            break

# damage_targets

In [4]:
train_id_damage_targets = []
train_id = []
import json
it = 0
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        mid_ar = []
        for i in record['damage_targets'].items():
            mid_ar.append(i)
            
        train_id_damage_targets.append(mid_ar)
        train_id.append(record['id'])

train_id_damage_targets_log = pd.DataFrame(train_id_damage_targets, index=train_id)
train_id_damage_targets = pd.DataFrame(index=train_id)

for train_id in tqdm_notebook(train_id_damage_targets_log.index):
    for i in train_id_damage_targets_log[train_id_damage_targets_log.index == train_id].values[0]:
        if i != None:
            train_id_damage_targets.at[train_id, '{}'.format(i[0])] = i[1]

KeyboardInterrupt: 

In [ ]:
test_id_damage_targets = []
test_id = []
import json
it = 0
with open('data/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        mid_ar = []
        for i in record['damage_targets'].items():
            mid_ar.append(i)
            
        test_id_damage_targets.append(mid_ar)
        test_id.append(record['id'])

test_id_damage_targets_log = pd.DataFrame(test_id_damage_targets, index=test_id)
test_id_damage_targets = pd.DataFrame(index=test_id)

for test_id in tqdm_notebook(test_id_damage_targets_log.index):
    for i in test_id_damage_targets_log[test_id_damage_targets_log.index == test_id].values[0]:
        if i != None:
            test_id_damage_targets.at[test_id, '{}'.format(i[0])] = i[1]

In [ ]:
train_id_damage_targets.fillna(0.01).to_csv('json_data/train_id_damage_targets.csv')
test_id_damage_targets.fillna(0.01).to_csv('json_data/test_id_damage_targets.csv')

# item_purchase_log, final_items

In [ ]:
train_id_items = []
train_id = []
import json
it = 0
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        mid_items = []
        for i in record['item_purchase_log']:
            mid_items.append(i['item_id'])
        train_id_items.append(mid_items)
        train_id.append(record['id'])
        
train_items_log = pd.DataFrame(train_id_items, index=train_id)
train_items_qual_count = pd.DataFrame(index=train_id)

for train_id in tqdm_notebook(train_items_log.index):
    x = train_items_log[train_items_log.index == train_id].values
    x = x[~np.isnan(x)]
    train_items_qual_count.at[train_id, 'x_len'] = len(x)
    qual_item = [items[items.index == i]['qual'].values[0] for i in x]
    for q_i in ['component', 'rare', 'epic', 'consumable', 'artifact', 'common', 'secret_shop']:
        train_items_qual_count.at[train_id, 'qual_item_{}_count'.format(q_i)] = qual_item.count(q_i)

In [ ]:
train_items_qual_count.astype(int).to_csv('json_data/train_items_qual_count.csv')

In [12]:
test_id_items = []
test_id = []
import json
it = 0
with open('data/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        mid_items = []
        for i in record['item_purchase_log']:
            mid_items.append(i['item_id'])
        test_id_items.append(mid_items)
        test_id.append(record['id'])

test_items_log = pd.DataFrame(test_id_items, index=test_id)
test_items_qual_count = pd.DataFrame(index=test_id)

for test_id in tqdm_notebook(test_items_log.index):
    x = test_items_log[test_items_log.index == test_id].values
    x = x[~np.isnan(x)]
    test_items_qual_count.at[test_id, 'x_len'] = len(x)
    qual_item = [items[items.index == i]['qual'].values[0] for i in x]
    for q_i in ['component', 'rare', 'epic', 'consumable', 'artifact', 'common', 'secret_shop']:
        test_items_qual_count.at[test_id, 'qual_item_{}_count'.format(q_i)] = qual_item.count(q_i)

In [13]:
test_items_qual_count.astype(int).to_csv('json_data/test_items_qual_count.csv')

<hr`>

In [14]:
train_id_items = []
train_id = []
import json
it = 0
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        train_id_items.append(record['final_items'])
        train_id.append(record['id'])
        
train_final_items_log = pd.DataFrame(train_id_items, index=train_id)
train_final_items_qual_and_cost = pd.DataFrame(index=train_id)

for train_id in tqdm_notebook(train_final_items_log.index):
    x = train_final_items_log[train_final_items_log.index == train_id].values[0]
    qual_item = [items[items.index == i]['qual'].values[0] for i in x if i != 0]
    sum_cost_items = sum([items[items.index == i]['cost'].values[0] for i in x if i != 0])
    
    train_final_items_qual_and_cost.at[train_id, 'final_items_sum'] = sum_cost_items
    for q_i in ['component', 'rare', 'epic', 'consumable', 'artifact', 'common', 'secret_shop']:
        train_final_items_qual_and_cost.at[train_id, 'final_items_{}'.format(q_i)] = qual_item.count(q_i)

In [15]:
test_id_items = []
test_id = []
import json
it = 0
with open('data/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        test_id_items.append(record['final_items'])
        test_id.append(record['id'])

test_final_items_log = pd.DataFrame(test_id_items, index=test_id)
test_final_items_qual_and_cost = pd.DataFrame(index=test_id)

for test_id in tqdm_notebook(test_final_items_log.index):
    x = test_final_items_log[test_final_items_log.index == test_id].values[0]
    qual_item = [items[items.index == i]['qual'].values[0] for i in x if i != 0]
    sum_cost_items = sum([items[items.index == i]['cost'].values[0] for i in x if i != 0])
    
    test_final_items_qual_and_cost.at[test_id, 'final_items_sum'] = sum_cost_items
    for q_i in ['component', 'rare', 'epic', 'consumable', 'artifact', 'common', 'secret_shop']:
        test_final_items_qual_and_cost.at[test_id, 'final_items_{}'.format(q_i)] = qual_item.count(q_i)

In [16]:
train_final_items_qual_and_cost['final_items_len'] = (train_final_items_qual_and_cost.iloc[:, -7:].sum(axis=1)).astype(int)
test_final_items_qual_and_cost['final_items_len'] = (test_final_items_qual_and_cost.iloc[:, -7:].sum(axis=1)).astype(int)
train_final_items_qual_and_cost.to_csv('json_data/train_final_items_qual_and_cost.csv')
test_final_items_qual_and_cost.to_csv('json_data/test_final_items_qual_and_cost.csv')

# series

In [17]:
train_series = []
train_id = []
import json
it = 0
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        p = np.array(record['series']['player_gold'])
        p_sum = p.sum()
        p_mean = p.mean()
        p_std = p.std()
        p_median = np.median(p)
        
        t = np.array(record['series']['dire_gold'])
        t_sum = t.sum()
        t_mean = t.mean()
        t_std = t.std()
        t_median = np.median(t)
        
        t1 = np.array(record['series']['radiant_gold'])
        t1_sum = t1.sum()
        t1_mean = t1.mean()
        t1_std = t1.std()
        t1_median = np.median(t1)
        
        
        delta_sum =  t_sum - p_sum
        delta_mean = t_mean - p_mean
        delta_std = t_std - p_std
        delta_median = t_median - p_median
        
        delta1_sum =  t1_sum - p_sum
        delta1_mean = t1_mean - p_mean
        delta1_std = t1_std - p_std
        delta1_median = t1_median - p_median
        
        
        train_id.append(record['id'])
        train_series.append([p_sum, p_mean, p_std, p_median, t_sum, t_mean, t_std, t_median, t1_sum, t1_mean, t1_std, t1_median, delta_sum, delta_mean, delta_std, delta_median, delta1_sum, delta1_mean, delta1_std, delta1_median])

In [18]:
train_series = pd.DataFrame(train_series, index=train_id)
train_series.columns = ['p_sum','p_mean','p_std','p_median','t_sum','t_mean','t_std','t_median', 't1_sum','t1_mean','t1_std','t1_median', 'delta_sum', 'delta_mean', 'delta_std', 'delta_median', 'delta1_sum', 'delta1_mean', 'delta1_std', 'delta1_median']
train_series.fillna(0).to_csv('json_data/train_series.csv')

In [19]:
test_series = []
test_id = []
import json
it = 0
with open('data/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        p = np.array(record['series']['player_gold'])
        p_sum = p.sum()
        p_mean = p.mean()
        p_std = p.std()
        p_median = np.median(p)
        
        t = np.array(record['series']['dire_gold'])
        t_sum = t.sum()
        t_mean = t.mean()
        t_std = t.std()
        t_median = np.median(t)
        
        t1 = np.array(record['series']['radiant_gold'])
        t1_sum = t1.sum()
        t1_mean = t1.mean()
        t1_std = t1.std()
        t1_median = np.median(t1)
        
        
        delta_sum =  t_sum - p_sum
        delta_mean = t_mean - p_mean
        delta_std = t_std - p_std
        delta_median = t_median - p_median
        
        delta1_sum =  t1_sum - p_sum
        delta1_mean = t1_mean - p_mean
        delta1_std = t1_std - p_std
        delta1_median = t1_median - p_median
        
        
        test_id.append(record['id'])
        test_series.append([p_sum, p_mean, p_std, p_median, t_sum, t_mean, t_std, t_median, t1_sum, t1_mean, t1_std, t1_median, delta_sum, delta_mean, delta_std, delta_median, delta1_sum, delta1_mean, delta1_std, delta1_median])

test_series = pd.DataFrame(test_series, index=test_id)
test_series.columns = ['p_sum','p_mean','p_std','p_median','t_sum','t_mean','t_std','t_median', 't1_sum','t1_mean','t1_std','t1_median', 'delta_sum', 'delta_mean', 'delta_std', 'delta_median', 'delta1_sum', 'delta1_mean', 'delta1_std', 'delta1_median']
test_series.fillna(0).to_csv('json_data/test_series.csv')

# level up times

In [20]:
train_id_items = []
train_id = []
import json
it = 0
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        m = np.mean(record['level_up_times'])
        s = np.std(record['level_up_times'])
        med = np.median(record['level_up_times'])
        ma =np.max(record['level_up_times'])
        
        train_id_items.append([m,s,med,ma])
        train_id.append(record['id'])
        
df_train = pd.DataFrame(train_id_items, index=train_id)
df_train.columns = ['l_u_mean', 'l_u_std', 'l_u_med', 'l_u_ma']
df_train.to_csv('json_data/l_u_train.csv')

In [21]:
test_id_items = []
test_id = []
import json
it = 0
with open('data/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        m = np.mean(record['level_up_times'])
        s = np.std(record['level_up_times'])
        med = np.median(record['level_up_times'])
        ma =np.max(record['level_up_times'])
        
        test_id_items.append([m,s,med,ma])
        test_id.append(record['id'])

df_test = pd.DataFrame(test_id_items, index=test_id)
df_test.columns = ['l_u_mean', 'l_u_std', 'l_u_med', 'l_u_ma']
df_test.to_csv('json_data/l_u_test.csv')

# radiant_heroes, dire_heroes, heroes

In [18]:
import requests

hero_id_2_winrate = {}
for hero in requests.get('https://api.opendota.com/api/heroStats').json():
    tmp = 0
    for i in range(1, 8):
        tmp += hero['{}_win'.format(i)] / hero['{}_pick'.format(i)]
    hero_id_2_winrate[hero['id']] = tmp / 7

    
winrate = [hero_id_2_winrate[hero_id] for hero_id in heroes.index]
heroes["winrate"] = winrate

In [9]:
from sklearn.preprocessing import OneHotEncoder

In [10]:
train_teammates = []
train_id = []
import json
it = 0
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        train_id.append(record['id'])
        train_teammates.append(record['{}_heroes'.format(record['player_team'])])

In [11]:
train_teammates_id = pd.DataFrame(train_teammates, index=train_id)
train_teammates= pd.DataFrame(index=train_id)

In [25]:
h0 = OneHotEncoder().fit_transform(train_teammates_id[0].values.reshape(-1,1))
h1 = OneHotEncoder().fit_transform(train_teammates_id[1].values.reshape(-1,1))
h2 = OneHotEncoder().fit_transform(train_teammates_id[2].values.reshape(-1,1))
h3 = OneHotEncoder().fit_transform(train_teammates_id[3].values.reshape(-1,1))
h4 = OneHotEncoder().fit_transform(train_teammates_id[4].values.reshape(-1,1))

In [26]:
df_h0 = pd.DataFrame(h0.toarray(), index=train_id)
df_h1 = pd.DataFrame(h1.toarray(), index=train_id)
df_h2 = pd.DataFrame(h2.toarray(), index=train_id)
df_h3 = pd.DataFrame(h3.toarray(), index=train_id)
df_h4 = pd.DataFrame(h4.toarray(), index=train_id)

In [27]:
train_teammates = df_h0 + df_h1 + df_h2 + df_h3 + df_h4

In [28]:
train_teammates = train_teammates.astype('int')

In [29]:
train_teammates.to_csv('json_data/train_teammates.csv')

In [22]:
test_teammates = []
test_id = []
import json
it = 0
with open('data/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        test_id.append(record['id'])
        test_teammates.append(record['{}_heroes'.format(record['player_team'])])

In [23]:
test_teammates_id = pd.DataFrame(test_teammates, index=test_id)
test_teammates= pd.DataFrame(index=test_id)

In [32]:
h0 = OneHotEncoder().fit_transform(test_teammates_id[0].values.reshape(-1,1))
h1 = OneHotEncoder().fit_transform(test_teammates_id[1].values.reshape(-1,1))
h2 = OneHotEncoder().fit_transform(test_teammates_id[2].values.reshape(-1,1))
h3 = OneHotEncoder().fit_transform(test_teammates_id[3].values.reshape(-1,1))
h4 = OneHotEncoder().fit_transform(test_teammates_id[4].values.reshape(-1,1))

In [33]:
df_h0 = pd.DataFrame(h0.toarray(), index=test_id)
df_h1 = pd.DataFrame(h1.toarray(), index=test_id)
df_h2 = pd.DataFrame(h2.toarray(), index=test_id)
df_h3 = pd.DataFrame(h3.toarray(), index=test_id)
df_h4 = pd.DataFrame(h4.toarray(), index=test_id)

In [34]:
test_teammates = df_h0 + df_h1 + df_h2 + df_h3 + df_h4

In [35]:
test_teammates = test_teammates.astype('int')

In [36]:
test_teammates.to_csv('data/test_teammates.csv')

In [19]:
train_id = []
train_tm_stats = []
for p_id in tqdm_notebook(train_teammates_id.index):
    #tm_roles = np.array([0,0,0,0,0,0,0,0,0])
    tm_agi = []
    tm_attack_range = []
    tm_attack_rate = []
    tm_winrate = []
    
    for tm in train_teammates_id[train_teammates_id.index == p_id].values[0]:
        #tm_roles += heroes[heroes.index == tm].iloc[:,-9:].values[0].astype(int)
        tm_agi.append(heroes[heroes.index == tm]['agi_gain'].values[0])
        tm_attack_range.append(heroes[heroes.index == tm]['attack_range'].values[0])
        tm_attack_rate.append(heroes[heroes.index == tm]['attack_rate'].values[0])
        tm_winrate.append(heroes[heroes.index == tm]['winrate'].values[0])
    train_id.append(p_id)

    tm_agi_mean = np.array(tm_agi).mean()
    tm_attack_range_mean = np.array(tm_attack_range).mean()
    tm_attack_rate_mean = np.array(tm_attack_rate).mean()
    tm_winrate_mean = np.array(tm_winrate).mean()
    
    tm_agi_std = np.array(tm_agi).std()
    tm_attack_range_std = np.array(tm_attack_range).std()
    tm_attack_rate_std = np.array(tm_attack_rate).std()
    tm_winrate_std = np.array(tm_winrate).std()
    
#     r0 = tm_roles[0]
#     r1 = tm_roles[1]
#     r2 = tm_roles[2]
#     r3 = tm_roles[3]
#     r4 = tm_roles[4]
#     r5 = tm_roles[5]
#     r6 = tm_roles[6]
#     r7 = tm_roles[7]
#     r8 = tm_roles[8]
    
    train_tm_stats.append([tm_agi_mean, tm_attack_range_mean, tm_attack_rate_mean,
                          tm_winrate_mean, tm_agi_std, tm_attack_range_std, tm_attack_rate_std, tm_winrate_std]) #, r0,r1,r2,r3,r4,r5,r6,r7,r8])

In [20]:
train_tm_stats = pd.DataFrame(train_tm_stats, index= train_id)
train_tm_stats.to_csv('json_data/train_tm_stats.csv')

In [24]:
test_id = []
test_tm_stats = []
for p_id in tqdm_notebook(test_teammates_id.index):
    tm_roles = np.array([0,0,0,0,0,0,0,0,0])
    tm_agi = []
    tm_attack_range = []
    tm_attack_rate = []
    tm_winrate = []
    
    for tm in test_teammates_id[test_teammates_id.index == p_id].values[0]:
        #tm_roles += heroes[heroes.index == tm].iloc[:,-9:].values[0].astype(int)
        tm_agi.append(heroes[heroes.index == tm]['agi_gain'].values[0])
        tm_attack_range.append(heroes[heroes.index == tm]['attack_range'].values[0])
        tm_attack_rate.append(heroes[heroes.index == tm]['attack_rate'].values[0])
        tm_winrate.append(heroes[heroes.index == tm]['winrate'].values[0])
    test_id.append(p_id)

    tm_agi_mean = np.array(tm_agi).mean()
    tm_attack_range_mean = np.array(tm_attack_range).mean()
    tm_attack_rate_mean = np.array(tm_attack_rate).mean()
    tm_winrate_mean = np.array(tm_winrate).mean()
    
    tm_agi_std = np.array(tm_agi).std()
    tm_attack_range_std = np.array(tm_attack_range).std()
    tm_attack_rate_std = np.array(tm_attack_rate).std()
    tm_winrate_std = np.array(tm_winrate).std()
    
#     r0 = tm_roles[0]
#     r1 = tm_roles[1]
#     r2 = tm_roles[2]
#     r3 = tm_roles[3]
#     r4 = tm_roles[4]
#     r5 = tm_roles[5]
#     r6 = tm_roles[6]
#     r7 = tm_roles[7]
#     r8 = tm_roles[8]
    
    test_tm_stats.append([tm_agi_mean, tm_attack_range_mean, tm_attack_rate_mean,
                          tm_winrate_mean, tm_agi_std, tm_attack_range_std, tm_attack_rate_std, tm_winrate_std]) #r0,r1,r2,r3,r4,r5,r6,r7,r8])

In [25]:
test_tm_stats = pd.DataFrame(test_tm_stats, index= test_id)

In [26]:
test_tm_stats.to_csv('json_data/test_tm_stats.csv')